# Importing Libraries

In [1]:
import pandas as pd
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import os
import tensorflow as tf
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction =0.6
session = tf.compat.v1.Session(config=config)

# Reading Data

In [57]:
train = pd.read_csv(r'C:\Users\acer\Desktop\NAG\Deep-Learning-case-study\HAR\UCI_HAR_Dataset\csv_files\train.csv')
test = pd.read_csv(r'C:\Users\acer\Desktop\NAG\Deep-Learning-case-study\HAR\UCI_HAR_Dataset\csv_files\test.csv')
print(train.shape, test.shape)


(7352, 564) (2947, 564)


# Converting labels to numeric Vector

In [58]:
labels=['LAYING', 'SITTING','STANDING','WALKING','WALKING_DOWNSTAIRS','WALKING_UPSTAIRS']
dict_classes={}
for i in range(6):
    dict_classes[labels[i]]=i

In [59]:
train.replace({"ActivityName":dict_classes},inplace=True)
test.replace({"ActivityName":dict_classes},inplace=True)

In [60]:
train.head()

,tBodyAccmeanX,tBodyAccmeanY,tBodyAccmeanZ,tBodyAccstdX,tBodyAccstdY,tBodyAccstdZ,tBodyAccmadX,tBodyAccmadY,tBodyAccmadZ,tBodyAccmaxX,...,angletBodyAccMeangravity,angletBodyAccJerkMeangravityMean,angletBodyGyroMeangravityMean,angletBodyGyroJerkMeangravityMean,angleXgravityMean,angleYgravityMean,angleZgravityMean,subject,Activity,ActivityName
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,5,2
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,5,2
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,5,2
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,5,2
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,5,2


In [62]:
# get X_train and y_train from csv files
X_train = train.drop(['subject', 'Activity', 'ActivityName'], axis=1)
y_train = train.ActivityName

In [63]:
# get X_test and y_test from test csv file
X_test = test.drop(['subject', 'Activity', 'ActivityName'], axis=1)
y_test = test.ActivityName

In [64]:
y_train=tf.keras.utils.to_categorical(y_train)
y_test=tf.keras.utils.to_categorical(y_test)

In [65]:
y_test.shape

(2947, 6)

# Simple Sequential Model

In [69]:

def create_model1():
  return tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(561),name="input-layer"),
    tf.keras.layers.Dense(16, activation='relu',name="dense1",kernel_initializer=tf.keras.initializers.glorot_normal()),
    tf.keras.layers.Dense(32, activation='relu',name="dense2",kernel_initializer=tf.keras.initializers.glorot_normal()),
    tf.keras.layers.Dense(64, activation='relu',name="dense3",kernel_initializer=tf.keras.initializers.glorot_normal()),
    tf.keras.layers.Dense(32, activation='relu',name="dense4",kernel_initializer=tf.keras.initializers.glorot_normal()),
    tf.keras.layers.Dense(16, activation='relu',name="dense5",kernel_initializer=tf.keras.initializers.glorot_normal()),
    tf.keras.layers.Dense(6, activation='softmax',name="output-layer")
  ])

In [70]:
#model1
model = create_model1()
#filepath="C:/Users/acer/Desktop/model_save1/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
#checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,monitor='val_accuracy',verbose=1, mode='auto',save_best_only=True,save_weights_only=True)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])


In [72]:
#lis=[tensorboard_callback,score]
history=model.fit(x=X_train, 
          y=y_train, 
          epochs=50,
          validation_data=(X_test, y_test))

Epoch 1/50
230/230 [==============================] - 1s 6ms/step - loss: 0.0689 - accuracy: 0.9742 - val_loss: 0.3557 - val_accuracy: 0.9016
Epoch 2/50
230/230 [==============================] - 1s 6ms/step - loss: 0.0699 - accuracy: 0.9721 - val_loss: 0.1577 - val_accuracy: 0.9450
Epoch 3/50
230/230 [==============================] - 1s 6ms/step - loss: 0.0668 - accuracy: 0.9752 - val_loss: 0.2032 - val_accuracy: 0.9328
Epoch 4/50
230/230 [==============================] - 1s 6ms/step - loss: 0.0538 - accuracy: 0.9808 - val_loss: 0.1393 - val_accuracy: 0.9508
Epoch 5/50
230/230 [==============================] - 1s 6ms/step - loss: 0.0541 - accuracy: 0.9797 - val_loss: 0.1771 - val_accuracy: 0.9430
Epoch 6/50
230/230 [==============================] - 1s 6ms/step - loss: 0.0493 - accuracy: 0.9812 - val_loss: 0.1636 - val_accuracy: 0.9498
Epoch 7/50
230/230 [==============================] - 1s 6ms/step - loss: 0.0763 - accuracy: 0.9735 - val_loss: 0.1681 - val_accuracy: 0.9460
Epoch 